In [12]:
from statsmodels.tsa.statespace.sarimax import SARIMAX
from statsmodels.stats.diagnostic import acorr_ljungbox
from sklearn.metrics import mean_absolute_error, mean_squared_error
import numpy as np
import pandas as pd
from pmdarima import auto_arima

# Step: Load data and enforce frequency
series = pd.read_csv(r"C:\Users\claud\OneDrive\Desktop\Dr Gu\negative_rate_series.csv", index_col='Date', parse_dates=True)
series = series.asfreq('M')

# Step: Check the data
print(series.head())
print(series.index)     # Should be DatetimeIndex
print(series.columns)   # Should be ['NegativeRate']

# Step: ARIMA: without seasonal component
arima_model = auto_arima(
    series['NegativeRate'],           
    seasonal=False,                   # no seasonality
    start_p=0, start_q=0,
    max_p=4, max_q=4,
    d=1,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=False)
print("Best ARIMA model:")
print(arima_model.summary())

# Step: SARIMA: with seasonal component (m=12 → monthly)
sarima_model = auto_arima(
    series['NegativeRate'],           
    seasonal=True,                    # enables seasonality
    m=12,                             # monthly seasonality
    start_p=0, start_q=0,
    max_p=3, max_q=3,
    start_P=0, start_Q=0,
    max_P=2, max_Q=2,
    d=1, D=1,
    trace=True,
    error_action='ignore',
    suppress_warnings=True,
    stepwise=False)

print("Best SARIMA model:")
print(sarima_model.summary())


C:\Users\claud\AppData\Local\Temp\ipykernel_28408\2085147236.py:10: FutureWarning: 'M' is deprecated and will be removed in a future version, please use 'ME' instead.
  series = series.asfreq('M')


            NegativeRate
Date                    
2003-09-30      1.000000
2003-10-31      0.052632
2003-11-30      0.075472
2003-12-31      0.000000
2004-01-31      0.063830
DatetimeIndex(['2003-09-30', '2003-10-31', '2003-11-30', '2003-12-31',
               '2004-01-31', '2004-02-29', '2004-03-31', '2004-04-30',
               '2004-05-31', '2004-06-30',
               ...
               '2012-01-31', '2012-02-29', '2012-03-31', '2012-04-30',
               '2012-05-31', '2012-06-30', '2012-07-31', '2012-08-31',
               '2012-09-30', '2012-10-31'],
              dtype='datetime64[ns]', name='Date', length=110, freq='ME')
Index(['NegativeRate'], dtype='object')
 ARIMA(0,1,0)(0,0,0)[0] intercept   : AIC=-160.838, Time=0.02 sec
 ARIMA(0,1,1)(0,0,0)[0] intercept   : AIC=-183.123, Time=0.29 sec
 ARIMA(0,1,2)(0,0,0)[0] intercept   : AIC=-181.259, Time=0.19 sec
 ARIMA(0,1,3)(0,0,0)[0] intercept   : AIC=-175.404, Time=0.28 sec
 ARIMA(0,1,4)(0,0,0)[0] intercept   : AIC=-193.544, Time=

In [13]:
# Step: Train/test split
split_idx = int(len(series) * 0.7)
train = series['NegativeRate'][:split_idx]
test = series['NegativeRate'][split_idx:]
test_dates = series.index[split_idx:]


# Step: Define all models to compare
model_configs = {
    'ARIMA(0,1,4) [auto_arima]': {'order': (0,1,4), 'seasonal_order': (0, 0, 0, 0)},
    'ARIMA(1,1,1) [manual]': {'order': (1, 1, 1), 'seasonal_order': (0, 0, 0, 0)},
    'SARIMA(2,1,3)(0,1,0,12) [auto_sarima]': {'order': (2, 1, 3), 'seasonal_order': (0, 1, 0, 12)},
    'SARIMA(2,1,2)(1,1,1,12) [manual]': {'order': (2, 1, 2), 'seasonal_order': (1, 1, 1, 12)}}

# Step: Fit and evaluate models
results_dict = {}

for name, cfg in model_configs.items():
    print(f"\nFitting {name}...")
    model = SARIMAX(
        train,
        order=cfg['order'],
        seasonal_order=cfg['seasonal_order'],
        enforce_stationarity=False,
        enforce_invertibility=False)
    result = model.fit(disp=False)

    # Forecast
    forecast = result.get_forecast(steps=len(test))
    pred = forecast.predicted_mean

    # Metrics
    mae = mean_absolute_error(test, pred)
    mse = mean_squared_error(test, pred)
    rmse = np.sqrt(mse)
    ljung = acorr_ljungbox(result.resid, lags=[10], return_df=True)
    ljung_pvalue = ljung['lb_pvalue'].iloc[0]

    # Store
    results_dict[name] = {
        'AIC': result.aic,
        'MAE': mae,
        'MSE': mse,
        'RMSE': rmse,
        'Ljung-Box p-value': ljung_pvalue}

# Step: Create comparison table
metrics_df = pd.DataFrame(results_dict).T
metrics_df = metrics_df[['AIC', 'MAE', 'MSE', 'RMSE', 'Ljung-Box p-value']]

# Step: Display
display(metrics_df)



Fitting ARIMA(0,1,4) [auto_arima]...

Fitting ARIMA(1,1,1) [manual]...

Fitting SARIMA(2,1,3)(0,1,0,12) [auto_sarima]...


C:\Users\claud\anaconda3\Lib\site-packages\statsmodels\base\model.py:607: ConvergenceWarning: Maximum Likelihood optimization failed to converge. Check mle_retvals
  warnings.warn("Maximum Likelihood optimization failed to "



Fitting SARIMA(2,1,2)(1,1,1,12) [manual]...


,AIC,MAE,MSE,RMSE,Ljung-Box p-value
"ARIMA(0,1,4) [auto_arima]",-190.189321,0.046910,0.002498,0.049980,0.008063
"ARIMA(1,1,1) [manual]",-200.526056,0.045042,0.002353,0.048508,0.021486
"SARIMA(2,1,3)(0,1,0,12) [auto_sarima]",-135.675459,0.028918,0.001041,0.032262,0.052163
"SARIMA(2,1,2)(1,1,1,12) [manual]",-181.723202,0.016572,0.000533,0.023079,0.131858
